In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pydicom import dcmread
from tqdm.notebook import tqdm
import os
from lxml.etree import Element, SubElement, tostring
from xml.dom.minidom import parseString

In [2]:
def make_xml(xmin_tuple, ymin_tuple, xmax_tuple, ymax_tuple, image_name, dst):
    node_root = Element('annotation')

    node_filename = SubElement(node_root, 'filename')
    node_filename.text = image_name

    for i in range(len(xmin_tuple)):
        node_object = SubElement(node_root, 'object')
        node_object_name = SubElement(node_object, 'name')
        node_object_name.text = str("thyroid_nodule")
        node_bndbox = SubElement(node_object, 'bndbox')
        node_xmin = SubElement(node_bndbox, 'xmin')
        node_xmin.text = str(xmin_tuple[i])
        node_ymin = SubElement(node_bndbox, 'ymin')
        node_ymin.text = str(ymin_tuple[i])
        node_xmax = SubElement(node_bndbox, 'xmax')
        node_xmax.text = str(xmax_tuple[i])
        node_ymax = SubElement(node_bndbox, 'ymax')
        node_ymax.text = str(ymax_tuple[i])

    xml = tostring(node_root)
    dom = parseString(xml)
    with open(dst, 'wb') as f:
        f.write(dom.toprettyxml(indent='\t', encoding='utf-8'))

In [3]:
image_root_path = "C:/Users/AbaKir/Downloads/thyroid/data/"
image_path = []
for root, dirs, files in os.walk(image_root_path):
    for name in files:
        image_path.append(os.path.join(root, name))
        
mask_root_path = "C:/Users/AbaKir/Downloads/thyroid/groundtruth/"
mask_path = []
for root, dirs, files in os.walk(mask_root_path):
    for name in files:
        mask_path.append(os.path.join(root, name))

In [4]:
sequence_number = 0
pic_save_dir = "./train/datasets/JPEGImages/"
xml_save_dir = "./train/datasets/Annotations/"
for num in tqdm(range(len(mask_path))):
    dic = {}
    mask_raw = dcmread(mask_path[num]).pixel_array
    image_raw = dcmread(image_path[num]).pixel_array
    for i in range(image_raw.shape[0]):
        mask_i = mask_raw[i]
        last_row_max = 0
        last_column_max = 0
        num_change_j = 0
        num_change_k = 0
        xmin, xmax, ymin, ymax = (), (), (), ()
        for j in range(mask_raw.shape[1]):
            row_max = np.max(mask_i[j, :])
            if row_max != last_row_max:
                num_change_j += 1
                if row_max == 1 and last_row_max == 0:
                    ymin = (j,)
                elif row_max == 0 and last_row_max == 1:
                    ymax = (j,)
            last_row_max = row_max
        for k in range(mask_raw.shape[2]):
            column_max = np.max(mask_i[:, k])
            if column_max != last_column_max:
                num_change_k += 1
                if column_max == 1 and last_column_max == 0:
                    xmin = (k,)
                elif column_max == 0 and last_column_max == 1:
                    xmax = (k,)
            last_column_max = column_max
        if num_change_j not in [0, 2] or num_change_k not in [0, 2]:
            pass
        else:
            dic[str(sequence_number).zfill(6)] = (xmin, xmax, ymin, ymax, image_raw[i])
        sequence_number += 1
    for pic_name in dic.keys():
        image = dic[pic_name][4]
        image = image.reshape((image.shape[0], image.shape[1], 1))
        image = np.concatenate((image, image, image), 2)
        Image.fromarray(image).save(pic_save_dir + pic_name + ".jpg")
        make_xml(dic[pic_name][0], dic[pic_name][2], dic[pic_name][1], dic[pic_name][3], 
                 pic_name + ".jpg", xml_save_dir + pic_name + ".xml")

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
from train.voc_annotation import voc_annotation
voc_annotation()

Generate txt in ImageSets.
train and val size 13428
train size 12085
Generate txt in ImageSets done.
Generate train.txt and val.txt for train.
